## Forcasting Inflation with FB Prophet
This project uses BLS CPI data and Facebook's Prophet forasting model to tell the future!

In [57]:
# Imports
import numpy as np
import pandas as pd
from prophet import Prophet
from fredapi import Fred
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.express as px
import Secrets

fred = Fred(api_key=Secrets.FRED_API_KEY)


In [49]:
cpi_raw = fred.get_series("CPIAUCSL")
cpi_df = pd.DataFrame(cpi_raw, columns=["y"]).reset_index()
cpi_df["ds"] = cpi_df["index"]
cpi_df
#px.line(cpi_df,title="Consumer Price Index for All Urban Consumers: All Items in U.S. City Average",labels={"value":"CPI","index":"Time","variable":"BLS"})

,index,y,ds
0,1947-01-01,21.480,1947-01-01
1,1947-02-01,21.620,1947-02-01
2,1947-03-01,22.000,1947-03-01
3,1947-04-01,22.000,1947-04-01
4,1947-05-01,21.950,1947-05-01
...,...,...,...
899,2021-12-01,280.126,2021-12-01
900,2022-01-01,281.933,2022-01-01
901,2022-02-01,284.182,2022-02-01
902,2022-03-01,287.708,2022-03-01


## Prophet!

In [50]:
m = Prophet()

m.fit(cpi_df)
forecast = m.predict(cpi_df)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -12.456
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3733.04    0.00268172       2756.02      0.5002      0.5002      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3859.76    0.00282453       1260.51           1           1      233   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        3909.2    0.00593681       9438.68           1           1      340   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3928.77    0.00489764       811.861      0.9673      0.9673      448   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499          3938   0.000316551       723.852      0.9503      0.9503      558   
    Iter      log prob        ||dx||      ||grad||       alpha   

,ds,yhat,yhat_lower,yhat_upper
899,2021-12-01,267.657204,265.342918,270.251460
900,2022-01-01,268.092477,265.635979,270.899922
901,2022-02-01,268.544696,265.966948,270.858846
902,2022-03-01,268.893339,266.575471,271.337614
903,2022-04-01,269.281368,266.921997,271.740355


In [51]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
1264,2023-03-28
1265,2023-03-29
1266,2023-03-30
1267,2023-03-31
1268,2023-04-01


In [52]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1264,2023-03-28,274.719871,272.149317,277.316151
1265,2023-03-29,274.471780,271.641484,276.837979
1266,2023-03-30,274.196975,271.656850,276.941910
1267,2023-03-31,273.901484,271.341935,276.375787
1268,2023-04-01,273.591833,271.183587,276.021879


In [59]:
plot_plotly(m, forecast)